In [11]:
# Evaluate Summaries with ROUGE
from rouge_score import rouge_scorer
import pandas as pd

In [12]:
original_data = pd.read_csv('../data/raw/transcripts.csv')
summaries_data = pd.read_csv('../data/processed/summary_outputs.csv')

In [13]:
# Fill NaNs
original_data['transcript'] = original_data['transcript'].fillna('')
summaries_data['Generated_Summary'] = summaries_data['Generated_Summary'].fillna('')

In [14]:
# First 3 TED Talks from sample_texts
sampled_originals = original_data['transcript'].sample(3, random_state=42).tolist()

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Store results
rouge_results = []

In [15]:
# Loop over each model
for model_name in summaries_data['Model'].unique():
    model_subset = summaries_data[summaries_data['Model'] == model_name]

    for i, row in model_subset.iterrows():
        example_no = row['Example_No'] - 1  
        generated_summary = row['Generated_Summary']
        reference_text = sampled_originals[example_no][:1000]  

        # Compute ROUGE
        scores = scorer.score(reference_text, generated_summary)

        rouge_results.append({
            'Model': model_name,
            'Example_No': row['Example_No'],
            'ROUGE-1': round(scores['rouge1'].fmeasure, 4),
            'ROUGE-2': round(scores['rouge2'].fmeasure, 4),
            'ROUGE-L': round(scores['rougeL'].fmeasure, 4)
        })

In [16]:
# Convert results to DataFrame
rouge_df = pd.DataFrame(rouge_results)

In [17]:
# ✅ Show first few results
rouge_df.head(10)

Model  Example_No  ROUGE-1  ROUGE-2  ROUGE-L
0    T5-small           1   0.2476   0.1923   0.2095
1    T5-small           2   0.4171   0.3732   0.4171
2    T5-small           3   0.3291   0.2979   0.3038
3     T5-base           1   0.2740   0.2028   0.2557
4     T5-base           2   0.2624   0.1553   0.2262
5     T5-base           3   0.1925   0.1097   0.1674
6  DistilBART           1   0.3930   0.3612   0.3843
7  DistilBART           2   0.3786   0.3431   0.3786
8  DistilBART           3   0.4223   0.4096   0.4223

In [18]:
rouge_df.to_csv('../reports/rouge_scores.csv', index=False)